In [1]:
import boto3
import pandas as pd
from io import BytesIO

# Create S3 and SNS clients
s3 = boto3.client('s3')
sns = boto3.client('sns', region_name='us-east-1')  # Change region if necessary

# Define bucket and file names
bucket2 = 'testing-exp-data-parquet'
participants_data_file = 'participants_data.parquet'

# Download the participants_data.parquet file from S3
obj = s3.get_object(Bucket=bucket2, Key=participants_data_file)
data = obj['Body'].read()

# Load participants data into a DataFrame
participants_df = pd.read_parquet(BytesIO(data))

# Create SNS topic
sns_topic_name = 'egift-card-compensation-parquet'
response = sns.create_topic(Name=sns_topic_name)
sns_topic_arn = response['TopicArn']

print(f"SNS topic '{sns_topic_name}' created with ARN: {sns_topic_arn}")

# Subscribe participants to the SNS topic
for email in participants_df['EmailAddress']:
    try:
        sns.subscribe(
            TopicArn=sns_topic_arn,
            Protocol='email',
            Endpoint=email
        )
        print(f"Subscribed {email} to SNS topic {sns_topic_name}")
    except Exception as e:
        print(f"Error subscribing {email}: {e}")

print("All participants subscribed successfully.")


SNS topic 'egift-card-compensation-parquet' created with ARN: arn:aws:sns:us-east-1:888853202619:egift-card-compensation-parquet
Subscribed ymuhannah@gmail.com to SNS topic egift-card-compensation-parquet
Subscribed aliceluo@uchicago.edu to SNS topic egift-card-compensation-parquet
All participants subscribed successfully.
